In [1]:
from lohrasb.best_estimator import BaseModel
from optuna.pruners import HyperbandPruner
from optuna.samplers._tpe.sampler import TPESampler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import optuna
from sklearn.pipeline import Pipeline
from feature_engine.imputation import (
    CategoricalImputer,
    MeanMedianImputer
    )
from category_encoders import OrdinalEncoder
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score)
from lightgbm import *
from lohrasb.utils.metrics import f1_plus_tn


2023-07-02 18:21:08,629 :: matplotlib :: matplotlib data path: /Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/matplotlib/mpl-data
2023-07-02 18:21:08,637 :: matplotlib :: CONFIGDIR=/Users/hjavedani/.matplotlib
2023-07-02 18:21:08,640 :: matplotlib :: interactive is False
2023-07-02 18:21:08,641 :: matplotlib :: platform is darwin
2023-07-02 18:21:08,663 :: graphviz._tools :: deprecate positional args: graphviz.backend.piping.pipe(['renderer', 'formatter', 'neato_no_op', 'quiet'])
2023-07-02 18:21:08,665 :: graphviz._tools :: deprecate positional args: graphviz.backend.rendering.render(['renderer', 'formatter', 'neato_no_op', 'quiet'])
2023-07-02 18:21:08,668 :: graphviz._tools :: deprecate positional args: graphviz.backend.unflattening.unflatten(['stagger', 'fanout', 'chain', 'encoding'])
2023-07-02 18:21:08,669 :: graphviz._tools :: deprecate positional args: graphviz.backend.viewing.view(['quiet'])
2023-07-02 18:21:08,678 :: graphviz._tools :: deprecate positio

#### Example 1 : Use Adult Data Set (a classification problem)
  
https://archive.ics.uci.edu/ml/datasets/Adult

#### Part 1: Use BestModel in sklearn pipeline


In [2]:
urldata= "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
# column names
col_names=["age", "workclass", "fnlwgt" , "education" ,"education-num",
"marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week",
"native-country","label"
]
# read data
data = pd.read_csv(urldata,header=None,names=col_names,sep=',')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


#### Define labels


In [3]:
data.loc[data['label']=='<=50K','label']=0
data.loc[data['label']==' <=50K','label']=0

data.loc[data['label']=='>50K','label']=1
data.loc[data['label']==' >50K','label']=1

data['label']=data['label'].astype(int)

#### Train test split

In [4]:
X = data.loc[:, data.columns != "label"]
y = data.loc[:, data.columns == "label"]
y = y.values.ravel()


X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.33, random_state=42)

# for sample_weights
weights = np.ones(len(y_train))


#### Find feature types for later use

In [5]:
int_cols =  X_train.select_dtypes(include=['int']).columns.tolist()
float_cols =  X_train.select_dtypes(include=['float']).columns.tolist()
cat_cols =  X_train.select_dtypes(include=['object']).columns.tolist()


#### Define model and set it argumens 

In [6]:
estimator = LGBMClassifier()
estimator_params = {
        "boosting_type":["gbdt"],
        "max_depth": [6,15],
        "learning_rate":[0.01, 0.1],
        "random_state":[42],

    }

kwargs = {  # params for fit method  
            'fit_optuna_kwargs' :{
            'sample_weight':None,
            },
            # params for OptunaSearch
            'main_optuna_kwargs' : {
            'estimator':estimator,
            'estimator_params':estimator_params,
            'refit':True,
            'measure_of_accuracy' :'f1_score(y_true, y_pred,average="weighted")',

            },
            'train_test_split_kwargs':{
                'test_size':.3,
                            
            },
            'study_search_kwargs':{
                'storage':None,
                'sampler':TPESampler(),
                'pruner':HyperbandPruner(),
                'study_name':"example of optuna optimizer",
                'direction':"maximize",
                'load_if_exists':False,
            },
            'optimize_kwargs':{
                # optuna optimization params
                'n_trials':20,
                'timeout':600,
                'catch':(),
                'callbacks':None,
                'gc_after_trial':False,
                'show_progress_bar':False,
            }
}




In [7]:
obj = BaseModel().optimize_by_optuna(
            kwargs=kwargs
        )

#### Build sklearn pipeline

In [8]:


pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model
            ('obj', obj),


 ])
 



#### Run Pipeline

In [9]:
pipeline.fit(X_train,y_train)
y_preds = pipeline.predict(X_test)
pred_labels = np.rint(y_preds)




[I 2023-07-02 18:21:11,048] A new study created in memory with name: example of optuna optimizer
[I 2023-07-02 18:21:11,562] Trial 0 finished with value: 0.8681205507217163 and parameters: {'boosting_type': 'gbdt', 'max_depth': 9, 'learning_rate': 0.09771716263805878, 'random_state': 42}. Best is trial 0 with value: 0.8681205507217163.
[I 2023-07-02 18:21:11,706] Trial 1 finished with value: 0.8679727712622133 and parameters: {'boosting_type': 'gbdt', 'max_depth': 10, 'learning_rate': 0.05258585856607267, 'random_state': 42}. Best is trial 0 with value: 0.8681205507217163.
[I 2023-07-02 18:21:11,824] Trial 2 finished with value: 0.8663337842850438 and parameters: {'boosting_type': 'gbdt', 'max_depth': 8, 'learning_rate': 0.04247910563725476, 'random_state': 42}. Best is trial 0 with value: 0.8681205507217163.
[I 2023-07-02 18:21:12,039] Trial 3 finished with value: 0.8561543696389076 and parameters: {'boosting_type': 'gbdt', 'max_depth': 11, 'learning_rate': 0.01973906770048023, 'rando

2023-07-02 18:21:14,353 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset, i.e., X_train and y_train!
2023-07-02 18:21:14,353 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset, i.e., X_train and y_train!


#### Check performance of the pipeline

In [10]:
print('F1 score : ')
print(f1_score(y_test,pred_labels))
print('Classification report : ')
print(classification_report(y_test,pred_labels))
print('Confusion matrix : ')
print(confusion_matrix(y_test,pred_labels))


F1 score : 
0.7114607697214544
Classification report : 
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      8196
           1       0.78      0.66      0.71      2550

    accuracy                           0.87     10746
   macro avg       0.84      0.80      0.82     10746
weighted avg       0.87      0.87      0.87     10746

Confusion matrix : 
[[7716  480]
 [ 877 1673]]


#### Some estimators have predict_proba method as well

In [11]:
y_preds = pipeline.predict_proba(X_test)
print(y_preds)

[[0.99212836 0.00787164]
 [0.59039043 0.40960957]
 [0.37888514 0.62111486]
 ...
 [0.64678902 0.35321098]
 [0.71025037 0.28974963]
 [0.97560267 0.02439733]]


#### Part 2:  Use BestModel as a standalone estimator 

In [12]:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.33, random_state=42)


#### Transform features to make them ready for model input

In [13]:
transform_pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model

 ])

#### Transform X_train and X_test

In [14]:
X_train=transform_pipeline.fit_transform(X_train,y_train)
X_test=transform_pipeline.transform(X_test)


#### Train model and predict

In [15]:
obj.fit(X_train,y_train)
y_pred = obj.predict(X_test)

[I 2023-07-02 18:21:15,057] A new study created in memory with name: example of optuna optimizer
[I 2023-07-02 18:21:15,278] Trial 0 finished with value: 0.867584249183425 and parameters: {'boosting_type': 'gbdt', 'max_depth': 13, 'learning_rate': 0.04993921347175169, 'random_state': 42}. Best is trial 0 with value: 0.867584249183425.
[I 2023-07-02 18:21:15,423] Trial 1 finished with value: 0.8685783630222276 and parameters: {'boosting_type': 'gbdt', 'max_depth': 13, 'learning_rate': 0.06204160472743217, 'random_state': 42}. Best is trial 1 with value: 0.8685783630222276.
[I 2023-07-02 18:21:15,718] Trial 2 finished with value: 0.8692705489562349 and parameters: {'boosting_type': 'gbdt', 'max_depth': 12, 'learning_rate': 0.05981864413429835, 'random_state': 42}. Best is trial 2 with value: 0.8692705489562349.
[I 2023-07-02 18:21:15,861] Trial 3 finished with value: 0.8657076757769573 and parameters: {'boosting_type': 'gbdt', 'max_depth': 14, 'learning_rate': 0.04504319241555171, 'rando

2023-07-02 18:21:18,244 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset, i.e., X_train and y_train!
2023-07-02 18:21:18,244 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset, i.e., X_train and y_train!


#### Check performance of the model

In [16]:
f1_plus_tn
print('F1 score plus TN : ')
print(f1_plus_tn(y_test,pred_labels))
print('F1 score : ')
print(f1_score(y_test,pred_labels))
print('Classification report : ')
print(classification_report(y_test,pred_labels))
print('Confusion matrix : ')
print(confusion_matrix(y_test,pred_labels))

F1 score plus TN : 
7716.711460769721
F1 score : 
0.7114607697214544
Classification report : 
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      8196
           1       0.78      0.66      0.71      2550

    accuracy                           0.87     10746
   macro avg       0.84      0.80      0.82     10746
weighted avg       0.87      0.87      0.87     10746

Confusion matrix : 
[[7716  480]
 [ 877 1673]]


In [17]:
obj.get_best_estimator()

LGBMClassifier(learning_rate=0.08354828503007254, max_depth=10, random_state=42)

In [18]:
obj.best_estimator

LGBMClassifier(learning_rate=0.08354828503007254, max_depth=10, random_state=42)

#### Get fitted randomized search object and its attributes

In [19]:
OptunaObj = obj.get_optimized_object()
print(OptunaObj)

FrozenTrial(number=19, state=TrialState.COMPLETE, values=[0.8708139662681024], datetime_start=datetime.datetime(2023, 7, 2, 18, 21, 18, 112704), datetime_complete=datetime.datetime(2023, 7, 2, 18, 21, 18, 243102), params={'boosting_type': 'gbdt', 'max_depth': 10, 'learning_rate': 0.08354828503007254, 'random_state': 42}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'boosting_type': CategoricalDistribution(choices=('gbdt',)), 'max_depth': IntDistribution(high=15, log=False, low=6, step=1), 'learning_rate': FloatDistribution(high=0.1, log=False, low=0.01, step=None), 'random_state': IntDistribution(high=42, log=False, low=42, step=1)}, trial_id=19, value=None)
